In [2]:
!pip install langchain
!pip install langchain_community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 50.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 20.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 411.6/411.6 kB 28.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 3.5 MB/s eta 0:00:00
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.3.25
    Uninstalling langchain-core-0.3.25:
      Successfully uninstalled langchain-core-0.3.25
  Attempting uninstall: langchain
    Found existing installation: langchain 0.3.12
    Uninstalling langchain-0.3.12:
      Successfully uninstalled langchain-0.3.12


In [13]:
import random
import pandas as pd
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.chat_models import ChatOpenAI

def create_prompt(names_df, surnames_df, provinces_df, vdc_df, zones, districts_df):
    person1 = names_df.sample().iloc[0]
    person2 = names_df.sample().iloc[0]
    while person1['name'] == person2['name']:
        person2 = names_df.sample().iloc[0]

    surname1 = surnames_df.sample().iloc[0]['Surname']
    surname2 = surnames_df.sample().iloc[0]['Surname']
    province1 = provinces_df.sample().iloc[0]
    province2 = provinces_df.sample().iloc[0]
    vdc = vdc_df.sample().iloc[0]
    zone = zones.sample().iloc[0]['zones']
    district = districts_df.sample().iloc[0]

    prompt = f"""
    Generate a unique sentence based on the following data:

    1. Names: {person1['name']} (gender: {person1['gender']}), {person2['name']} (gender: {person2['gender']})
    2. Surnames: {surname1}, {surname2}
    3. Provinces: {province1['name']} (headquarter: {province1['headquarter']}), {province2['name']} (headquarter: {province2['headquarter']})
    4. District: {district['names']} (headquarter: {district['headquarters']})
    5. VDC/Municipality: {vdc['name']} ({vdc['status']}) in {vdc['district']} district
    6. Zone: {zone}

    Conditions:
    - Use proper pronouns based on gender.
    - Include a mix of personal, geographical, and relational elements.
    - Ensure the sentence is grammatically correct and unique.

    Example:
    "{person1['name']} {surname1} lives in {province1['headquarter']}, the headquarter of {province1['name']}. {'She' if person1['gender'] == 'girl' else 'He'} often visits {vdc['name']} ({vdc['status']}) in {vdc['district']} district. {'Her' if person1['gender'] == 'girl' else 'His'} friend {person2['name']} {surname2} lives in {district['headquarters']}, a district headquarter in {district['names']}, located in {zone} zone."

    Now generate a completely new sentence based on these data points and the structure of each sentence should be completely unique.
    """
    return prompt.strip()

names_df = pd.read_csv("/content/popular_nepali_names.csv")
provinces_df = pd.read_csv("/content/provience_names_and_headquarters.csv")
vdc_df = pd.read_csv("/content/vdc_municipality.csv")
districts_df = pd.read_csv("/content/district_names_and_headquarters.csv")
surnames_df = pd.read_csv("/content/popular_nepali_surnames.csv")
zones = pd.read_csv("/content/zones.csv")

llm = ChatOpenAI(openai_api_key="api key", model="gpt-4", temperature=0.7)

# Prepare the prompt template
prompt_template = PromptTemplate(
    input_variables=[],
    template="{text}"
)

chain = LLMChain(llm=llm, prompt=prompt_template)

used_names = set()
used_surnames = set()
unique_sentences = []

for _ in range(1000):
    prompt = create_prompt(names_df, surnames_df, provinces_df, vdc_df, zones, districts_df)
    response = chain.run(text=prompt)
    unique_sentences.append(response)

    used_names.update([prompt.split("Names:")[1].split("(gender")[0].strip(), prompt.split("Names:")[1].split(",")[1].split("(gender")[0].strip()])
    used_surnames.update([prompt.split("Surnames:")[1].split(",")[0].strip(), prompt.split("Surnames:")[1].split(",")[1].strip()])

missing_names = set(names_df['name']) - used_names
missing_surnames = set(surnames_df['Surname']) - used_surnames
# if missing_names or missing_surnames:
#     raise ValueError(f"Not all names or surnames were used: Missing names: {missing_names}, Missing surnames: {missing_surnames}")

# Save the generated sentences to a file
with open("unique_sentences.txt", "w") as f:
    for sentence in unique_sentences:
        f.write(sentence + "\n")

print("Generated 1000 unique sentences and saved to unique_sentences.txt")



RateLimitError: Error code: 429 - {'error': {'message': 'You exceeded your current quota, please check your plan and billing details. For more information on this error, read the docs: https://platform.openai.com/docs/guides/error-codes/api-errors.', 'type': 'insufficient_quota', 'param': None, 'code': 'insufficient_quota'}}

In [15]:
# Verify all names and surnames are used
missing_names = set(names_df['name']) - used_names
missing_surnames = set(surnames_df['Surname']) - used_surnames
# if missing_names or missing_surnames:
#     raise ValueError(f"Not all names or surnames were used: Missing names: {missing_names}, Missing surnames: {missing_surnames}")

# Save the generated sentences to a file
with open("unique_sentences.txt", "w") as f:
    for sentence in unique_sentences:
        f.write(sentence + "\n")

print("Generated 1000 unique sentences and saved to unique_sentences.txt")


Generated 1000 unique sentences and saved to unique_sentences.txt


In [16]:
import pandas as pd
with open('/content/unique_sentences.txt', 'r') as file:
    sentences = file.readlines()

sentences = [sentence.strip() for sentence in sentences if sentence.strip()]

df = pd.DataFrame(sentences, columns=["Sentence"])

# Save to a CSV file
df.to_csv('output.csv', index=False)


#using gemeni flash


In [18]:
pip install -q -U google-generativeai

In [20]:
import google.generativeai as genai

genai.configure(api_key="ur api key")

model = genai.GenerativeModel("gemini-1.5-flash")
response = model.generate_content("How does AI work?")
print(response.text)

AI works by combining large amounts of data with fast, iterative processing and intelligent algorithms, allowing the software to learn automatically from patterns or features in the data.  There's no single way AI works, as it encompasses a broad range of techniques, but here's a breakdown of some key aspects:

**1. Data:**  AI systems are trained on massive datasets.  The more data, the better the AI typically performs. This data can be anything from images and text to sensor readings and financial transactions.  The quality and relevance of the data are crucial for accurate and reliable results.

**2. Algorithms:**  These are sets of rules and statistical techniques that allow the AI to learn from the data.  Different types of AI use different algorithms:

* **Machine Learning (ML):** This is a subset of AI where systems learn from data without explicit programming.  Instead of being explicitly programmed with rules, ML algorithms identify patterns and relationships in the data to ma

In [21]:
import random
import pandas as pd
import google.generativeai as genai

genai.configure(api_key="api key")

def create_prompt(names_df, surnames_df, provinces_df, vdc_df, zones, districts_df):
    person1 = names_df.sample().iloc[0]
    person2 = names_df.sample().iloc[0]
    while person1['name'] == person2['name']:
        person2 = names_df.sample().iloc[0]

    surname1 = surnames_df.sample().iloc[0]['Surname']
    surname2 = surnames_df.sample().iloc[0]['Surname']
    province1 = provinces_df.sample().iloc[0]
    province2 = provinces_df.sample().iloc[0]
    vdc = vdc_df.sample().iloc[0]
    zone = zones.sample().iloc[0]['zones']
    district = districts_df.sample().iloc[0]

    prompt = f"""
    Generate a concise sentence using the following data:

    1. Names: {person1['name']} (gender: {person1['gender']}), {person2['name']} (gender: {person2['gender']})
    2. Surnames: {surname1}, {surname2}
    3. Provinces: {province1['name']} (headquarter: {province1['headquarter']}), {province2['name']} (headquarter: {province2['headquarter']})
    4. District: {district['names']} (headquarter: {district['headquarters']})
    5. VDC/Municipality: {vdc['name']} ({vdc['status']}) in {vdc['district']} district
    6. Zone: {zone}

    Generate a short sentence using the above details, ensuring it's no longer than 2-3 short sentences. Avoid long paragraphs and keep it simple.

    Example:
    "{person1['name']} {surname1} lives in {province1['headquarter']}, the headquarter of {province1['name']}. {'She' if person1['gender'] == 'girl' else 'He'} often visits {vdc['name']} in {vdc['district']} district, where {person2['name']} {surname2} lives."

    Now generate a completely new, concise sentence with a similar structure.
    """
    return prompt.strip()

names_df = pd.read_csv("/content/popular_nepali_names.csv")
provinces_df = pd.read_csv("/content/provience_names_and_headquarters.csv")
vdc_df = pd.read_csv("/content/vdc_municipality.csv")
districts_df = pd.read_csv("/content/district_names_and_headquarters.csv")
surnames_df = pd.read_csv("/content/popular_nepali_surnames.csv")
zones = pd.read_csv("/content/zones.csv")

used_names = set()
used_surnames = set()
unique_sentences = []

for _ in range(1000):
    prompt = create_prompt(names_df, surnames_df, provinces_df, vdc_df, zones, districts_df)

    # Use Gemini API to generate the sentence
    response = genai.GenerativeModel("gemini-1.5-flash").generate_content(prompt)
    unique_sentences.append(response.text)

    # Mark names and surnames as used
    used_names.update([prompt.split("Names:")[1].split("(gender")[0].strip(), prompt.split("Names:")[1].split(",")[1].split("(gender")[0].strip()])
    used_surnames.update([prompt.split("Surnames:")[1].split(",")[0].strip(), prompt.split("Surnames:")[1].split(",")[1].strip()])

# Verify all names and surnames are used
missing_names = set(names_df['name']) - used_names
missing_surnames = set(surnames_df['Surname']) - used_surnames
# if missing_names or missing_surnames:
#     raise ValueError(f"Not all names or surnames were used: Missing names: {missing_names}, Missing surnames: {missing_surnames}")

with open("unique_sentences_gemeni.txt", "w") as f:
    for sentence in unique_sentences:
        f.write(sentence + "\n")

print("Generated 1000 unique, concise sentences and saved to unique_sentences_gemeni.txt")


TooManyRequests: 429 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-1.5-flash:generateContent?%24alt=json%3Benum-encoding%3Dint: Resource has been exhausted (e.g. check quota).

In [22]:
import random
import pandas as pd
import google.generativeai as genai
import time
from requests.exceptions import HTTPError

genai.configure(api_key=" gemeni api key ")

def create_prompt(names_df, surnames_df, provinces_df, vdc_df, zones, districts_df):
    person1 = names_df.sample().iloc[0]
    person2 = names_df.sample().iloc[0]
    while person1['name'] == person2['name']:
        person2 = names_df.sample().iloc[0]

    surname1 = surnames_df.sample().iloc[0]['Surname']
    surname2 = surnames_df.sample().iloc[0]['Surname']
    province1 = provinces_df.sample().iloc[0]
    province2 = provinces_df.sample().iloc[0]
    vdc = vdc_df.sample().iloc[0]
    zone = zones.sample().iloc[0]['zones']
    district = districts_df.sample().iloc[0]

    prompt = f"""
    Generate a concise sentence using the following data:

    1. Names: {person1['name']} (gender: {person1['gender']}), {person2['name']} (gender: {person2['gender']})
    2. Surnames: {surname1}, {surname2}
    3. Provinces: {province1['name']} (headquarter: {province1['headquarter']}), {province2['name']} (headquarter: {province2['headquarter']})
    4. District: {district['names']} (headquarter: {district['headquarters']})
    5. VDC/Municipality: {vdc['name']} ({vdc['status']}) in {vdc['district']} district
    6. Zone: {zone}

    Generate a short sentence using the above details, ensuring it's no longer than 2-3 short sentences. Avoid long paragraphs and keep it simple.

    Example:
    "{person1['name']} {surname1} lives in {province1['headquarter']}, the headquarter of {province1['name']}. {'She' if person1['gender'] == 'girl' else 'He'} often visits {vdc['name']} in {vdc['district']} district, where {person2['name']} {surname2} lives."

    Now generate a completely new, concise sentence with a similar structure.
    """
    return prompt.strip()

# Function to handle retries in case of rate limiting
def generate_sentence_with_retry(prompt, retries=5, delay=10):
    for attempt in range(retries):
        try:
            response = genai.GenerativeModel("gemini-1.5-flash").generate_content(prompt)
            return response.text
        except HTTPError as e:
            if e.response.status_code == 429:
                print(f"Rate limit exceeded, retrying in {delay} seconds...")
                time.sleep(delay)  # wait before retrying
            else:
                raise  # re-raise if not a rate limit error
    raise Exception("Max retries exceeded")

names_df = pd.read_csv("/content/popular_nepali_names.csv")
provinces_df = pd.read_csv("/content/provience_names_and_headquarters.csv")
vdc_df = pd.read_csv("/content/vdc_municipality.csv")
districts_df = pd.read_csv("/content/district_names_and_headquarters.csv")
surnames_df = pd.read_csv("/content/popular_nepali_surnames.csv")
zones = pd.read_csv("/content/zones.csv")

# Track usage of all names and surnames
used_names = set()
used_surnames = set()
unique_sentences = []

for _ in range(1000):
    prompt = create_prompt(names_df, surnames_df, provinces_df, vdc_df, zones, districts_df)

    # Use Gemini API with retry mechanism
    try:
        response = generate_sentence_with_retry(prompt)
        unique_sentences.append(response)

        # Mark names and surnames as used
        used_names.update([prompt.split("Names:")[1].split("(gender")[0].strip(), prompt.split("Names:")[1].split(",")[1].split("(gender")[0].strip()])
        used_surnames.update([prompt.split("Surnames:")[1].split(",")[0].strip(), prompt.split("Surnames:")[1].split(",")[1].strip()])
    except Exception as e:
        print(f"Error generating sentence: {e}")
        continue

missing_names = set(names_df['name']) - used_names
missing_surnames = set(surnames_df['Surname']) - used_surnames
# if missing_names or missing_surnames:
#     raise ValueError(f"Not all names or surnames were used: Missing names: {missing_names}, Missing surnames: {missing_surnames}")


with open("unique_sentences.txt", "w") as f:
    for sentence in unique_sentences:
        f.write(sentence + "\n")

print("Generated 1000 unique, concise sentences and saved to unique_sentences.txt")


Error generating sentence: 429 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-1.5-flash:generateContent?%24alt=json%3Benum-encoding%3Dint: Resource has been exhausted (e.g. check quota).


Error generating sentence: 429 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-1.5-flash:generateContent?%24alt=json%3Benum-encoding%3Dint: Resource has been exhausted (e.g. check quota).


Error generating sentence: 429 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-1.5-flash:generateContent?%24alt=json%3Benum-encoding%3Dint: Resource has been exhausted (e.g. check quota).


Error generating sentence: 429 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-1.5-flash:generateContent?%24alt=json%3Benum-encoding%3Dint: Resource has been exhausted (e.g. check quota).


Error generating sentence: 429 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-1.5-flash:generateContent?%24alt=json%3Benum-encoding%3Dint: Resource has been exhausted (e.g. check quota).


...

Error generating sentence: 429 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-1.5-flash:generateContent?%24alt=json%3Benum-encoding%3Dint: Resource has been exhausted (e.g. check quota).


Error generating sentence: 429 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-1.5-flash:generateContent?%24alt=json%3Benum-encoding%3Dint: Resource has been exhausted (e.g. check quota).


Error generating sentence: 429 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-1.5-flash:generateContent?%24alt=json%3Benum-encoding%3Dint: Resource has been exhausted (e.g. check quota).


Error generating sentence: 429 POST https://generativelanguage.googleapis.com/v1beta/models/gemini-1.5-flash:generateContent?%24alt=json%3Benum-encoding%3Dint: Resource has been exhausted (e.g. check quota).


KeyboardInterrupt: 